# Load Data and preprocess

In [1]:
import sys
import os
sys.path.append(os.path.abspath(".."))

In [2]:
#pip install ucimlrepo

In [3]:
from src.data.load_data import load_input_data
from src.data.load_data import preprocess_data

In [4]:
X_train, X_test, y_train, y_test, metadata, variables, datasetdf = load_input_data()

In [10]:
import pandas as pd

num_cols = ['Attribute2', 'Attribute5', 'Attribute8', 'Attribute11',
            'Attribute13', 'Attribute16', 'Attribute18']

X_train, X_test = preprocess_data(X_train, X_test, num_cols)
#pd.set_option('display.max_columns', None)
#print(X_train.head())

     Attribute2  Attribute5  Attribute8  Attribute11  Attribute13  \
10    -0.733512   -0.713001    0.054714    -1.660121    -0.956361   
82    -0.231598   -0.610869    0.054714     1.076342    -1.047717   
827   -0.231598    0.360687   -0.835976    -0.747967     0.048549   
410    0.270317   -0.461601    0.945404     1.076342    -1.413139   
48    -0.817165    1.506577   -1.726666     1.076342     0.322615   

     Attribute16  Attribute18  Attribute1_A12  Attribute1_A13  Attribute1_A14  \
10     -0.724565    -0.434114               1               0               0   
82     -0.724565    -0.434114               0               0               1   
827     1.074000     2.303542               0               0               1   
410    -0.724565    -0.434114               1               0               0   
48      1.074000    -0.434114               0               0               1   

     Attribute3_A31  Attribute3_A32  Attribute3_A33  Attribute3_A34  \
10                0        

# Train model

In [6]:
from src.model.train_model import train_classifier

In [7]:
clf = train_classifier(X_train,y_train,n_estimators=100, random_state=42)

# Test model

In [8]:
from src.model.test_model import test_classifier_prediction

In [9]:
y_pred,classification_report,accuracy_score = test_classifier_prediction(clf,X_test,y_test)

In [10]:
print(accuracy_score)

0.76


In [11]:
print(classification_report)

              precision    recall  f1-score   support

           1       0.76      0.95      0.85       210
           2       0.74      0.31      0.44        90

    accuracy                           0.76       300
   macro avg       0.75      0.63      0.64       300
weighted avg       0.76      0.76      0.72       300


### User constraints / Ontology constraints

In [6]:
# Implement here


### Final outputs of Input section of pipeline
For experimentation, here are some values

In [ ]:
from src.utilities.desired_space import desired_space
x= X_test[0] #test instance
p= {
    'Attribute5':(10000, 30000) #credit amount
}#perturbation map
desired_space= desired_space(2, datasetdf)#
cat_f= X_train.select_dtypes(include=['bool']) #categorical features
num_f= X_train.select_dtypes(include=['int64', 'float64']) #numerical features
protect_f= ['Attribute9_A92'  'Attribute9_A93'  'Attribute9_A94'] # Personal status and sex (X_train.head() shows only 3 values (columns) while there are 5) #protected features
features= X_train.columns() #all features
t= 2 #desired outcome
f= clf #black-box model for which we want to find CF
X= X_train #training data
step= {
    'Attribute5': 10 #credit amount steps
} #dictionary holding the feature distribution to be used for single feature perturbation method

# UFCE

### Feature Selection: Mutual Information (MI)

In [2]:
from itertools import combinations
from src.utilities.mutual_information import mi_score
from src.utilities.mutual_information import MI
features = list(X_train.columns)
feature_pairs = list(combinations(features, 2))
print(feature_pairs[:5])
mi_list = MI(feature_pairs, X_train) #used all features available #original authors do over entire X but it is not preprocessed... Which i think is nessecary? 
# top 5 
print(mi_list[:5])

ModuleNotFoundError: No module named 'src'

### Nearest Neighbour: KD Tree

In [ ]:
from src.utilities.nearest_neighbours import FNN
